Importing Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

Using mediapipe for keypoint detection

In [2]:
# Holistic model
mp_holistic = mp.solutions.holistic 

# Drawing utilities
mp_drawing = mp.solutions.drawing_utils 

In [3]:
# The image is a frame generated using opencv and to use this frame in mediapipe the frame need to be in RGB
# The frames from opencv is in BGR

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert image from BGR to RGB
    image.flags.writeable = False                  # For locking the features of Image 
    results = model.process(image)                 # Use image in mediapipe for making prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert image from RGB to BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

saving the array

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['Hello','Thank You','Please','Father','What','Help','Go','Finish','Busy','More','Morning'])
actions = np.array(['Hello'])
# Sixty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 60

In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Collecting data for training and testing

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic_model)


                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv2.waitKey(10) & 0xFF == 27:
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()

Data preprocessing and creating labels

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'Hello': 0,
 'Thank You': 1,
 'Please': 2,
 'Father': 3,
 'What': 4,
 'Help': 5,
 'Go': 6,
 'Finish': 7,
 'Busy': 8,
 'More': 9,
 'Morning': 10}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences)

In [13]:
y = to_categorical(labels).astype(int)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

LSTM training

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
# actions[np.argmax(res)]

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=80, callbacks=[tb_callback])

Epoch 1/80
19/19 [==============================] - 8s 122ms/step - loss: 2.7158 - categorical_accuracy: 0.0976
Epoch 2/80
19/19 [==============================] - 2s 127ms/step - loss: 2.2222 - categorical_accuracy: 0.1684
Epoch 3/80
19/19 [==============================] - 2s 112ms/step - loss: 2.1574 - categorical_accuracy: 0.1818
Epoch 4/80
19/19 [==============================] - 2s 119ms/step - loss: 1.7829 - categorical_accuracy: 0.3519
Epoch 5/80
19/19 [==============================] - 2s 112ms/step - loss: 1.7193 - categorical_accuracy: 0.3468
Epoch 6/80
19/19 [==============================] - 2s 103ms/step - loss: 1.3693 - categorical_accuracy: 0.4697
Epoch 7/80
19/19 [==============================] - 2s 114ms/step - loss: 1.3502 - categorical_accuracy: 0.4714
Epoch 8/80
19/19 [==============================] - 2s 114ms/step - loss: 1.2456 - categorical_accuracy: 0.5488
Epoch 9/80
19/19 [==============================] - 2s 121ms/step - loss: 1.9882 - categorical_accuracy:

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 11)                363       
                                                                 
Total params: 596,939
Trainable params: 596,939
Non-trai

Prediction

In [22]:
res = model.predict(X_test)

3/3 [==============================] - 1s 27ms/step


In [26]:
actions[np.argmax(res[5])]

'Father'

In [27]:
model.save('Sign_language_model.h5')

In [29]:
model.load_weights('Sign_language_model.h5')

Saving Model

Evaluation using Confusion Matrix and Accuracy

In [30]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [31]:
yhat = model.predict(X_test)

3/3 [==============================] - 0s 27ms/step


In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[60,  0],
        [ 0,  6]],

       [[61,  0],
        [ 0,  5]],

       [[60,  0],
        [ 0,  6]],

       [[54,  0],
        [ 0, 12]],

       [[56,  4],
        [ 0,  6]],

       [[62,  0],
        [ 1,  3]],

       [[61,  0],
        [ 4,  1]],

       [[59,  0],
        [ 0,  7]],

       [[62,  0],
        [ 1,  3]],

       [[59,  2],
        [ 0,  5]],

       [[60,  0],
        [ 0,  6]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

0.9090909090909091

Test in real Time

In [31]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        # #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 34ms/step
Father
1/1 [==============================] - 0s 32ms/step
Father
1/1 [==============================] - 0s 37ms/step
Father
1/1 [==============================] - 0s 32ms/step
Father
1/1 [==============================] - 0s 33ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 33ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 37ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 32ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 34ms/step
Father
1/1 [==============================] - 0s 33ms/step
Father
1/1 [==============================] - 0s 35ms/step
Fath

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)]

0.164487

In [ ]:
res[np.argmax(res)] > threshold

True